# Entorno Remoto de Compilación (MambaVision) 🚀

Este notebook se ejecuta enteramente en los servidores de Google Colab conectado desde tu VS Code.
Sigue los pasos secuenciales para descargar tu código desde GitHub y compilar el Kernel Custom de C++ contra los headers de TensorFlow provistos por Google.

In [7]:
import os
import sys

# ==========================================
# CONFIGURACIÓN DE TU REPOSITORIO DE GITHUB
# ==========================================
GITHUB_REPO = "https://github.com/rdromer2/mambavision_tf_kernel.git"
REPO_NAME = "mambavision_tf_kernel"
PROJECT_PATH = f"/content/{REPO_NAME}"

# 1. Clonar o actualizar el repositorio
if not os.path.exists(PROJECT_PATH):
    print(f"Clonando el repositorio {GITHUB_REPO}...")
    os.system(f'git clone {GITHUB_REPO} {PROJECT_PATH}')
else:
    print("El repositorio ya existe. Actualizando con git pull...")
    os.system(f'cd {PROJECT_PATH} && git pull')

# 2. Añadir al sys.path
sys.path.append(PROJECT_PATH)

# 3. Cambiar nuestro directorio activo al proyecto
os.chdir(PROJECT_PATH)
print(f"\nDirectorio de trabajo actual: {os.getcwd()}")

El repositorio ya existe. Actualizando con git pull...

Directorio de trabajo actual: /content/mambavision_tf_kernel


## 🛠️ Compilación C++ delegada en CMake
Delegamos integramente el proceso de construcción en CMake en la subcarpeta `build/`, lo que garantiza mayor mantenibilidad y asegura que las aserciones formales y el fix de ABI establecidos en `CMakeLists.txt` surtan efecto sistemáticamente.

In [12]:
import os

print("Limpiando compilaciones anteriores...")
# El comando de purga se inyecta aquí, antes de crear la carpeta de nuevo
!rm -rf {PROJECT_PATH}/build

print("Configurando y Compilando el Kernel C++ y CUDA...")
!mkdir -p {PROJECT_PATH}/build
%cd {PROJECT_PATH}/build
!cmake ..
!make -j
%cd {PROJECT_PATH}

print("\n✅ Compilación finalizada.")

Limpiando compilaciones anteriores...
Configurando y Compilando el Kernel C++ y CUDA...
/content/mambavision_tf_kernel/build
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- The CUDA compiler identification is NVIDIA 12.8.93 with host compiler GNU 11.4.0
-- Detecting CUDA compiler ABI info
-- Detecting CUDA compiler ABI info - done
-- Check for working CUDA compiler: /usr/local/cuda/bin/nvcc - skipped
-- Detecting CUDA compile features
-- Detecting CUDA compile features - done
-- Found Python3: /usr/local/bin/python (found version "3.1

## 🧪 Prueba Inicial del Kernel (El Centinela de Memoria)
**Detalle Técnico**: Mantenemos el hack de `ctypes.RTLD_GLOBAL` vital porque seguimos utilizando Colab y los binarios de TF no están compilados estáticamente.
Esta prueba evaluará un flujo correcto (3D) y un caso de fallo intencionado (2D) para verificar la intercepción del centinela de memoria.

In [13]:
import ctypes
import tensorflow as tf
import numpy as np
import glob
import os

# 1. EL PARCHE ABI (Estrictamente necesario en Colab)
tf_lib_dir = tf.sysconfig.get_lib()
fw_libs = glob.glob(os.path.join(tf_lib_dir, 'libtensorflow_framework.so*'))
if fw_libs:
    lib_path = sorted(fw_libs)[-1]
    print(f'Precargando RTLD_GLOBAL: {lib_path}')
    ctypes.CDLL(lib_path, ctypes.RTLD_GLOBAL)

try:
    # 2. CARGA DEL KERNEL (Ruta corregida hacia la carpeta build)
    mamba_module = tf.load_op_library(f'{PROJECT_PATH}/build/libmamba_kernel.so')
    
    # 3. PREPARACIÓN DE LA MEMORIA (Tensor 3D de unos)
    input_data = np.ones((2, 4, 8), dtype=np.float32)
    
    print("\nIniciando transferencia a VRAM y ejecución CUDA...")
    # Forzar la ejecución física en el chip de NVIDIA
    with tf.device('/GPU:0'):
        input_tensor = tf.constant(input_data)
        # Aquí cruzamos el puente: Python -> C++ -> CUDA
        output_tensor = mamba_module.mamba_selective_scan(input_tensor)
        
    # 4. EXTRACCIÓN Y VALIDACIÓN
    resultado = output_tensor.numpy()
    print("Tensor entrada (muestra):", input_data.flatten()[:5])
    print("Tensor salida  (muestra):", resultado.flatten()[:5])
    
    # Validación estricta: Comprobar si todos los elementos se multiplicaron por 2
    validacion = np.allclose(resultado, input_data * 2.0)
    print(f"\nEstado de validación matemática (x * 2.0): {validacion}")

except Exception as e:
    print("\nFallo en la ejecución interceptado:")
    print(e)

Precargando RTLD_GLOBAL: /usr/local/lib/python3.12/dist-packages/tensorflow/libtensorflow_framework.so.2

Iniciando transferencia a VRAM y ejecución CUDA...
Tensor entrada (muestra): [1. 1. 1. 1. 1.]
Tensor salida  (muestra): [1. 1. 1. 1. 1.]

Estado de validación matemática (x * 2.0): False
